In [2]:
#In diesem File wird das Modell für den Klassifier definiert und trainiert
import tensorflow.keras as keras
from tensorflow.keras import preprocessing
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D, MaxPool2D,Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
tf.config.threading.set_intra_op_parallelism_threads(2)
tf.config.threading.set_inter_op_parallelism_threads(2)


In [5]:
#Image Data Generator zum trainieren des models wird definiert

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2, #Shear Range und Zoom um das Modell auch auf nicht aufrecht stehende Daten vorzubereiten
    zoom_range = 0.2,
    validation_split = 0.25
)

data_path='./data/train/'
train_set = train_datagen.flow_from_directory(
    data_path,
    target_size = (28, 28),
    color_mode = 'grayscale',
    batch_size = 128*4,
    class_mode = 'categorical',
    shuffle = True,
    subset='training'
)
valid_set = train_datagen.flow_from_directory(
    data_path,
    target_size = (28, 28),
    color_mode = 'grayscale',
    batch_size = 128*4,
    class_mode = 'categorical',
    shuffle = True,
    subset='validation'
)

Found 187611 images belonging to 12 classes.
Found 62530 images belonging to 12 classes.


In [ ]:
with tf.device('/CPU:0'):

    #Model inpired from https://www.kaggle.com/yassineghouzam/introduction-to-cnn-keras-0-997-top-6
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',
                     activation ='relu', input_shape = (28,28,1)))

    model.add(MaxPool2D(pool_size=(4,4)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',
                     activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))


    model.add(Flatten())
    model.add(Dense(64, activation = "relu"))
    model.add(Dense(12, activation = "softmax"))

    # compile model
    adam = tf.keras.optimizers.Adam(lr = 5e-4)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])


    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)
    model.fit(train_set,validation_data=valid_set,epochs=20, verbose=1, batch_size=128, callbacks=[callback])

Epoch 1/20
367/367 [==============================] - 616s 2s/step - loss: 1.0154 - accuracy: 0.6718 - val_loss: 0.5723 - val_accuracy: 0.8236

In [ ]:
val_loss, val_accuracy = model.evaluate(valid_set)
print(val_loss,val_accuracy)

In [6]:
model.save('./models/Digit_Recognizer/')

INFO:tensorflow:Assets written to: ./models/Digit_Recognizer/assets


In [29]:
#Load model to test if it worked correctly

tf.keras.models.load_model('./models/Digit_Recognizer/')

In [71]:
#Test if Prediction Works
def prediction(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    plt.imshow(img, cmap = 'gray')
    img = cv2.resize(img,(40, 40))
    norm_image = cv2.normalize(img, None, alpha = 0, beta = 1, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    #norm_image=img/255
    norm_image = norm_image.reshape((norm_image.shape[0], norm_image.shape[1], 1))
    case = np.asarray([norm_image])
    pred = np.argmax(model.predict([case]), axis=-1)
    return ([i for i in train_set.class_indices if train_set.class_indices[i]==(pred[0])][0],pred)

In [5]:
image = cv2.imread('././././data/singlenumber.JPG')

prediction(image)

NameError: name 'prediction' is not defined

In [47]:
train_set.class_indices

{'+': 0,
 '-': 1,
 '0': 2,
 '1': 3,
 '2': 4,
 '3': 5,
 '4': 6,
 '5': 7,
 '6': 8,
 '7': 9,
 '8': 10,
 '9': 11}